In [1]:
# imports 
import pandas as pd
import numpy as np
import json
import boto3
from io import StringIO

In [2]:
s3 = boto3.client('s3')
BUCKET_NAME = 'data-quest-bucket-rearc'
S3_PREFIX = "raw/bls/"

In [3]:
s3 = boto3.client('s3')
bucket = 'data-quest-bucket-rearc'
bls_key = 'raw/bls/pr.data.0.Current'
api_data_key = 'raw/census_bureau/population_data.json'

# Fetch the object from S3
bls_response = s3.get_object(Bucket=bucket, Key=bls_key)
api_data_response = s3.get_object(Bucket=bucket, Key=api_data_key)
# Read the content as a string
bls_data = bls_response['Body'].read().decode('utf-8')
api_data = api_data_response['Body'].read().decode('utf-8')

bls_dtypes = {
    'series_id': str,
    'year': int,
    'period': str,
    'value': float,
    'footnote_codes': str
}

# Load into Pandas
df_bls = pd.read_csv(StringIO(bls_data), sep='\t',dtype=bls_dtypes)
df_api = pd.DataFrame(json.loads(api_data)['data'])

In [4]:
print(f"The schema of the bls dataframe is: {df_bls.dtypes}")
print(df_bls.head(10))

The schema of the bls dataframe is: series_id                str
year                   int64
period                   str
       value         float64
footnote_codes           str
dtype: object
   series_id          year period         value footnote_codes
0  PRS30006011        1995    Q01           2.6            NaN
1  PRS30006011        1995    Q02           2.1            NaN
2  PRS30006011        1995    Q03           0.9            NaN
3  PRS30006011        1995    Q04           0.1            NaN
4  PRS30006011        1995    Q05           1.4            NaN
5  PRS30006011        1996    Q01          -0.2            NaN
6  PRS30006011        1996    Q02          -0.3            NaN
7  PRS30006011        1996    Q03          -0.1            NaN
8  PRS30006011        1996    Q04           0.2            NaN
9  PRS30006011        1996    Q05          -0.1            NaN


In [5]:
print(f"The schema of the census bureau api dataframe is: {df_bls.dtypes}")
print(df_api.head(10))

The schema of the census bureau api dataframe is: series_id                str
year                   int64
period                   str
       value         float64
footnote_codes           str
dtype: object
  Nation ID         Nation  Year   Population
0   01000US  United States  2013  316128839.0
1   01000US  United States  2014  318857056.0
2   01000US  United States  2015  321418821.0
3   01000US  United States  2016  323127515.0
4   01000US  United States  2017  325719178.0
5   01000US  United States  2018  327167439.0
6   01000US  United States  2019  328239523.0
7   01000US  United States  2021  331893745.0
8   01000US  United States  2022  333287562.0
9   01000US  United States  2023  334914896.0


In [6]:
# CLEANING: Trim column names and string values
df_bls.columns = df_bls.columns.str.strip()

# Strip whitespace from all string columns at once
df_bls = df_bls.apply(lambda x: x.str.strip() if isinstance(x, str) else x)

# errors='coerce' turns non-numeric junk (like empty strings or text) into NaN
df_bls['value'] = pd.to_numeric(df_bls['value'], errors='coerce')

# 2. Fill all NaNs (including those created above) with 0
df_bls['value'] = df_bls['value'].fillna(0)

# 3. (Optional) Convert to a specific type if needed, like float
df_bls['value'] = df_bls['value'].astype(float)

In [7]:
# Population statistics for 2013 - 2018 inclusive
# To Calculate the mean and standard deviation of the US population.

mask = (df_api['Year'] >= 2013) & (df_api['Year'] <= 2018)
pop_subset = df_api.loc[mask, 'Population']

mean_pop = pop_subset.mean()
std_pop = pop_subset.std()

print(f"Mean Population [2013-2018]: {mean_pop:,.2f}")
print(f"Standard Deviation: {std_pop:,.2f}")

Mean Population [2013-2018]: 322,069,808.00
Standard Deviation: 4,158,441.04


In [8]:
# Best Year per Series
# Find the year with the highest total value for each series_id.

df_yearly = df_bls.groupby(['series_id', 'year'])['value'].sum().reset_index()

df_best_years = df_yearly.sort_values('value', ascending=False) \
                         .drop_duplicates('series_id') \
                         .sort_values('series_id')

print("Best Year per Series (Sample):")
print(df_best_years.head())

Best Year per Series (Sample):
             series_id  year    value
27   PRS30006011        2022   20.500
58   PRS30006012        2022   17.100
65   PRS30006013        1998  705.895
108  PRS30006021        2010   17.700
139  PRS30006022        2010   12.400


In [57]:
# 1. Filter BLS for the target series and period
df_target_bls = df_bls[(df_bls['series_id'] == 'PRS30006032') & (df_bls['period'] == 'Q01')]

# 2. Merge (Join) with the Population dataframe
# We use an 'inner' join on the year columns
report_df = pd.merge(
    df_target_bls, 
    df_api, 
    left_on='year', 
    right_on='Year', 
    how='inner'
)

# 3. Select only the required columns
final_report = report_df[['series_id', 'year', 'period', 'value', 'Population']]

print("Final Combined Report:")
print(final_report.sort_values('year'))

Final Combined Report:
     series_id  year period  value   Population
0  PRS30006032  2013    Q01    0.5  316128839.0
1  PRS30006032  2014    Q01   -0.1  318857056.0
2  PRS30006032  2015    Q01   -1.7  321418821.0
3  PRS30006032  2016    Q01   -1.4  323127515.0
4  PRS30006032  2017    Q01    0.9  325719178.0
5  PRS30006032  2018    Q01    0.5  327167439.0
6  PRS30006032  2019    Q01   -1.6  328239523.0
7  PRS30006032  2021    Q01    0.7  331893745.0
8  PRS30006032  2022    Q01    5.3  333287562.0
9  PRS30006032  2023    Q01    0.3  334914896.0
